In [1]:
import pandas as pd
import numpy as np
import re as re
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import folium
import googlemaps
from datetime import datetime
import os
from dotenv import load_dotenv
load_dotenv()

def getPage(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [2]:
url = "http://www.dutchrental.com/commercial-property/Amsterdam"
soup = getPage(url)
soup

<!DOCTYPE HTML>

<html lang="en">
<head>
<meta charset="utf-8"/>
<title>Commercial property for rent Amsterdam</title>
<meta content="Copyright (c) 2011 oezoezoe software / mijnhuren.nl" name="Copyright"/>
<meta content="Industial unit, retail outlet, office space, store, office, business premises, hotel, commercial property, restaurant, storage space, garage" name="keywords"/>
<meta content="Commercial property for rent in Amsterdam. Industrial units, stores, retail outlets, offices and restaurants in Amsterdam. Offered by professional Estate brokers." name="description"/>
<meta content="ALL" name="robots"/>
<meta content="index,follow" name="robots"/>
<link href="/Content/gfx/favicon.png" rel="icon" type="image/png"/>
<link href="http://www.dutchrental.com/commercial-property/Amsterdam" rel="canonical"/>
<meta content="VY2GKeUkTUWCkhSfZbPNqhORiJyy6bn_iptn0a3UjHA" name="google-site-verification"/>
<!--[if IE]>
<link rel="shortcut icon" href="/Content/gfx/favicon.ico" type="image/vnd.m

In [3]:
rlocal = soup.select('p a')
rlocal

[<a href="/commercial-property-for-rent/Amsterdam/Corsicaweg-10/770329/overview" title="Corsicaweg 10">
 <strong>Corsicaweg 10 Amsterdam</strong></a>,
 <a href="/commercial-property-for-rent/Amsterdam/Robert-Kochplantsoen-19/770331/overview" title="Robert Kochplantsoen 19">
 <strong>Robert Kochplantsoen 19 Amsterdam</strong></a>,
 <a href="/commercial-property-for-rent/Amsterdam/Eerste-Leliedwarsstraat-6/744638/overview" title="Eerste Leliedwarsstraat 6">
 <strong>Eerste Leliedwarsstraat 6 Amsterdam</strong></a>,
 <a href="/commercial-property-for-rent/Amsterdam/Geuzenstraat-98/744754/overview" title="Geuzenstraat 98">
 <strong>Geuzenstraat 98 Amsterdam</strong></a>,
 <a href="/commercial-property-for-rent/Amsterdam/Jan-Rebelstraat-18-C/739903/overview" title="Jan Rebelstraat 18 C">
 <strong>Jan Rebelstraat 18 C Amsterdam</strong></a>,
 <a href="/commercial-property-for-rent/Amsterdam/Willebroekstraat-62/733702/overview" title="Willebroekstraat 62">
 <strong>Willebroekstraat 62 Amsterd

In [4]:
web =[]
for r in rlocal:
    web.append(r.get('href'))
print(web)

['/commercial-property-for-rent/Amsterdam/Corsicaweg-10/770329/overview', '/commercial-property-for-rent/Amsterdam/Robert-Kochplantsoen-19/770331/overview', '/commercial-property-for-rent/Amsterdam/Eerste-Leliedwarsstraat-6/744638/overview', '/commercial-property-for-rent/Amsterdam/Geuzenstraat-98/744754/overview', '/commercial-property-for-rent/Amsterdam/Jan-Rebelstraat-18-C/739903/overview', '/commercial-property-for-rent/Amsterdam/Willebroekstraat-62/733702/overview', '/commercial-property-for-rent/Amsterdam/Admiraal-de-Ruijterweg-402/650655/overview', '/commercial-property-for-rent/Amsterdam/Geuzenstraat-98/644697/overview']


In [5]:
web1=[]
for i in range (2,6):
    x = getPage(f"http://www.dutchrental.com/commercial-property/Amsterdam?page={i}")   
    rlocal1 = x.select('p a')
    for r in rlocal1:
        web1.append(r.get('href'))
print(web1)


['/commercial-property-for-rent/Amsterdam/Nieuw-Zeelandweg-6-E/571247/overview', '/commercial-property-for-rent/Amsterdam/Meentstraat-66/558209/overview', '/commercial-property-for-rent/Amsterdam/Johan-Huizingalaan-110/558606/overview', '/commercial-property-for-rent/Amsterdam/Keurenplein-17-A/515379/overview', '/commercial-property-for-rent/Amsterdam/Socratesstraat-26/466064/overview', '/commercial-property-for-rent/Amsterdam/Stadhouderskade-55-parkeerplaats/462061/overview', '/commercial-property-for-rent/Amsterdam/Rhoneweg-18/449573/overview', '/commercial-property-for-rent/Amsterdam/Stadhouderskade-55/408508/overview', '/commercial-property-for-rent/Amsterdam/Petroleumhavenweg-22/360434/overview', '/commercial-property-for-rent/Amsterdam/Licentiehouder-gezocht/358640/overview', '/commercial-property-for-rent/Amsterdam/Willemsparkweg-135-B/346519/overview', '/commercial-property-for-rent/Amsterdam/Contactweg-131/204353/overview', '/commercial-property-for-rent/Amsterdam/Hemonystraat

In [6]:
import re
cleaning = web+web1
clean =[]
for c in cleaning:
    c1 = re.sub("/commercial-property-for-rent/Amsterdam/", "", c)
    c2 = re.sub("\/[0-9]+\/overview", "", c1)
    c3 = re.sub("\-", " ", c2)
    clean.append(c3)
clean

['Corsicaweg 10',
 'Robert Kochplantsoen 19',
 'Eerste Leliedwarsstraat 6',
 'Geuzenstraat 98',
 'Jan Rebelstraat 18 C',
 'Willebroekstraat 62',
 'Admiraal de Ruijterweg 402',
 'Geuzenstraat 98',
 'Nieuw Zeelandweg 6 E',
 'Meentstraat 66',
 'Johan Huizingalaan 110',
 'Keurenplein 17 A',
 'Socratesstraat 26',
 'Stadhouderskade 55 parkeerplaats',
 'Rhoneweg 18',
 'Stadhouderskade 55',
 'Petroleumhavenweg 22',
 'Licentiehouder gezocht',
 'Willemsparkweg 135 B',
 'Contactweg 131',
 'Hemonystraat 66',
 'Nieuwe Hemweg 4 c',
 'Veemarkt 212 F',
 'Siriusdreef 17 27']

In [7]:
gmaps = googlemaps.Client(key=os.getenv("key"))


In [8]:
def coordLocals(listlocals):
    addresslocals = []
    for n in listlocals:
        addresslocals.append(gmaps.places(n))      
    return addresslocals

localams = coordLocals(clean)

In [21]:
localams[0]['results'][0]['name']

'Corsicaweg 10'

In [22]:
def getParameters(companies):
    param =[]
    for comp in companies:
        for i in range(len(comp['results'])):
            longitude = comp['results'][i]['geometry']['location']['lng']
            latitude = comp['results'][i]['geometry']['location']['lat']
            address = comp['results'][i]['name']
            loc = {
            'address': address,
            'coordinates':[longitude, latitude]
            }
            param.append(loc)
    return param

In [23]:
x = getParameters(localams)

In [24]:
x

[{'address': 'Corsicaweg 10', 'coordinates': [4.8115896, 52.4021632]},
 {'address': 'Robert Kochplantsoen 19',
  'coordinates': [4.9369403, 52.3480536]},
 {'address': 'Eerste Leliedwarsstraat 6',
  'coordinates': [4.882497799999999, 52.3759084]},
 {'address': 'Geuzenstraat 98', 'coordinates': [4.8648476, 52.37328549999999]},
 {'address': 'Jan Rebelstraat 18C', 'coordinates': [4.7859449, 52.3614967]},
 {'address': 'Willebroekstraat 62',
  'coordinates': [4.805356700000001, 52.344418]},
 {'address': 'Admiraal de Ruijterweg 402',
  'coordinates': [4.8512083, 52.3827785]},
 {'address': 'Geuzenstraat 98', 'coordinates': [4.8648476, 52.37328549999999]},
 {'address': 'Nieuw Zeelandweg 6', 'coordinates': [4.8013445, 52.4081418]},
 {'address': 'Meentstraat 66', 'coordinates': [4.7865597, 52.35511839999999]},
 {'address': 'Johan Huizingalaan 110',
  'coordinates': [4.826863299999999, 52.36132240000001]},
 {'address': 'Keurenplein 17', 'coordinates': [4.7844578, 52.3603337]},
 {'address': 'Socrat

In [26]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [27]:
db, coll = connectCollection('companies', 'companies')

In [28]:
col = db["rentlocals"]
col.insert_many(x)

In [29]:
def getLocation(companies):
    location =[]
    longitude = companies['coordinates'][1]
    latitude = companies['coordinates'][0]
    loc = {
        'type':'Point',
        'coordinates':[latitude, longitude]
    }
    location.append(loc)
    print(location)
    return location

In [32]:
x[0]

{'address': 'Corsicaweg 10',
 'coordinates': [4.8115896, 52.4021632],
 '_id': ObjectId('5dd69bd2775b3ed4af276077')}

In [36]:
db, collr = connectCollection('companies', 'rentlocals')


In [37]:
for x1 in x:
    value = {"$set": {'location':getLocation(x1)}}
    query = {"_id": x1['_id']}
    collr.update_many(query, value)

[{'type': 'Point', 'coordinates': [4.8115896, 52.4021632]}]
[{'type': 'Point', 'coordinates': [4.9369403, 52.3480536]}]
[{'type': 'Point', 'coordinates': [4.882497799999999, 52.3759084]}]
[{'type': 'Point', 'coordinates': [4.8648476, 52.37328549999999]}]
[{'type': 'Point', 'coordinates': [4.7859449, 52.3614967]}]
[{'type': 'Point', 'coordinates': [4.805356700000001, 52.344418]}]
[{'type': 'Point', 'coordinates': [4.8512083, 52.3827785]}]
[{'type': 'Point', 'coordinates': [4.8648476, 52.37328549999999]}]
[{'type': 'Point', 'coordinates': [4.8013445, 52.4081418]}]
[{'type': 'Point', 'coordinates': [4.7865597, 52.35511839999999]}]
[{'type': 'Point', 'coordinates': [4.826863299999999, 52.36132240000001]}]
[{'type': 'Point', 'coordinates': [4.7844578, 52.3603337]}]
[{'type': 'Point', 'coordinates': [5.940916199999999, 50.9309853]}]
[{'type': 'Point', 'coordinates': [5.891920499999999, 51.9466695]}]
[{'type': 'Point', 'coordinates': [4.8135102, 52.374663]}]
[{'type': 'Point', 'coordinates': 

In [38]:
%store x

Stored 'x' (list)


In [95]:
# Con esto compruebo si puedo hacer web scraping con selenium la primera o de manera normal como la segunda, 
# crea un archivo en la carpeta que estoy y si se abre en el navegador es que puedo hacerlo sin selenium

test = requests.get("https://www.fundainbusiness.nl/en/alle-bedrijfsaanbod/amsterdam/").text
with open("test.html","w+") as file:
    file.write(test)

    
test = requests.get("http://www.dutchrental.com/commercial-property/Amsterdam").text
with open("test.html","w+") as file:
    file.write(test)